In [ ]:
import numpy as np
import time
import numbers
import functools
# from Ipython.display import Image as Im
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
%matplotlib inline

def extract_elements(condition, x):
    if isinstance(x, numbers.Number):
        return x
    else:
        return np.extract(condition, x)


class clPoint3d():
    def __init__(self, x, y, z):
        (self.x, self.y, self.z) = (x, y, z)

    def __sub__(self, other_point):
        return clVector3d(self.x - other_point.x, self.y - other_point.y, self.z - other_point.z)

    def __add__(self, other_vector):
        return clPoint3d(self.x + other_vector.x, self.y + other_vector.y, self.z + other_vector.z)

    def get_components(self):
        return (self.x, self.y, self.z)

    def get_np_array(self):
        return np.array((self.x, self.y, self.z))

    def extract_elements(self, condition):
        return clVector3d(extract_elements(condition, self.x), extract_elements(condition, self.y),
                          extract_elements(condition, self.z))

    def replace_elements(self, condition):
        temp = clVector3d(np.zeros(condition.shape), np.zeros(condition.shape), np.zeros(condition.shape))
        np.place(temp.x, condition, self.x)
        np.place(temp.y, condition, self.y)
        np.place(temp.z, condition, self.z)
        return temp


class clVector3d():
    def __init__(self, x, y, z):
        (self.x, self.y, self.z) = (x, y, z)

    def __mul__(self, other_vector):
        return clVector3d(self.x * other_vector, self.y * other_vector, self.z * other_vector)

    def __add__(self, other_vector):
        return clVector3d(self.x + other_vector.x, self.y + other_vector.y, self.z + other_vector.z)

    def __sub__(self, other_vector):
        return clVector3d(self.x - other_vector.x, self.y - other_vector.y, self.z - other_vector.z)

    def dot(self, other_vector):
        return (self.x * other_vector.x) + (self.y * other_vector.y) + (self.z * other_vector.z)

    def mag(self):
        return np.linalg.norm(self)

    def normalize(self):
        magnitude = np.sqrt(self.dot(self))
        return self * (1.0 / np.where(magnitude == 0, 1, magnitude))

    def get_components(self):
        return (self.x, self.y, self.z)

    def get_np_array(self):
        return np.array((self.x, self.y, self.z))

    def cross(self, other_vector):
        temp = np.cross(self.get_np_array(), other_vector.get_np_array())
        return clVector3d(temp[0], temp[1], temp[2])

    def extract_elements(self, condition):
        return clVector3d(extract_elements(condition, self.x), extract_elements(condition, self.y),
                          extract_elements(condition, self.z))

    def replace_elements(self, condition):
        temp = clVector3d(np.zeros(condition.shape), np.zeros(condition.shape), np.zeros(condition.shape))
        np.place(temp.x, condition, self.x)
        np.place(temp.y, condition, self.y)
        np.place(temp.z, condition, self.z)
        return temp



class Sphere:
    def __init__(self, center, radius, diffuse, specular_coef=0.5, texture=None):
        self.center = center
        self.radius = radius
        self.diffuse = diffuse
        self.specular_coef = specular_coef
        self.texture = texture

    def intersect(self, ray_starting_point, normalized_ray_direction):
        eye_to_center = ray_starting_point - self.center
        b = 2 * normalized_ray_direction.dot(eye_to_center)
        c = eye_to_center.dot(eye_to_center) - (self.radius * self.radius)
        discriminant = (b ** 2) - (4 * c)
        discriminant_root = np.sqrt(np.maximum(0, discriminant))
        root1 = (-b - discriminant_root) / 2.0
        root2 = (-b + discriminant_root) / 2.0
        selected_root = np.where((root1 > 0) & (root1 < root2), root1, root2)
        return np.where((discriminant > 0) & (selected_root > 0), selected_root, infinity_plus)

    def diffuse_color(self, M=None):
        if self.texture:
            checker = ((M.x * 2).astype(int) % 2) == ((M.z * 2).astype(int) % 2)
            return self.diffuse * checker
        else:
            return self.diffuse

    def calculate_intensity(self, ray_starting_point, normalized_ray_direction, intersection, objects_list,
                            number_of_bounces):
        M = (ray_starting_point + normalized_ray_direction * intersection)  # intersection point
        N = (M - self.center) * (1. / self.radius)  # normal
        toL = (light_position - M).normalize()  # direction to light
        toO = (eye_position - M).normalize()  # direction to ray origin
        nudged = M + N * .0001  # M nudged to avoid itself

        # Shadow: find if the point is shadowed or not.
        # This amounts to finding out if M can see the light
        light_distances = [object.intersect(nudged, toL) for object in objects_list]
        light_nearest = functools.reduce(np.minimum, light_distances)
        seelight = light_distances[objects_list.index(self)] == light_nearest

        # Ambient
        color = rgb(0.05, 0.05, 0.05)

        # Lambert shading (diffuse)
        lv = np.maximum(N.dot(toL), 0)
        color += self.diffuse_color(M) * lv * seelight

        # Reflection
        if number_of_bounces < 5:
            rayD = (normalized_ray_direction - N * 2 * normalized_ray_direction.dot(N)).normalize()
            color += raytrace(nudged, rayD, objects_list, number_of_bounces + 1) * self.specular_coef

        # Blinn-Phong shading (specular)
        phong = N.dot((toL + toO).normalize())
        color += rgb(1, 1, 1) * np.power(np.clip(phong, 0, 1), 50) * seelight
        return color

class clCamera:
    def __init__(self, eye, look_at, up, field_of_view, resolution_width, resolution_height):
        self.eye = eye
        self.look_at = look_at
        self.up = up
        self.fov = field_of_view
        self.resolution_width = resolution_width
        self.resolution_height = resolution_height
        self.N = (self.look_at - self.eye).normalize()
        self.U = self.up.cross(self.N).normalize()
        # True up vector
        self.V = self.N.cross(self.U).normalize()
        self.aspect_ratio = self.resolution_width / self.resolution_height
        self.fov_rad = np.radians(field_of_view)
        self.half_width = np.tan(self.fov_rad / 2)
        self.half_height = self.half_width / self.aspect_ratio
        screen_corners = (
            -self.half_width, self.half_width / self.aspect_ratio, self.half_width,
            -self.half_width / self.aspect_ratio)
        x = np.tile(np.linspace(screen_corners[0], screen_corners[2], self.resolution_width), self.resolution_height)
        y = np.repeat(np.linspace(screen_corners[1], screen_corners[3], self.resolution_height), self.resolution_width)
        image_center = self.eye + self.N  # Assuming image plane is 1 unit away
        self.pixels_world_coordinates = (
                image_center.get_np_array().reshape(-1, 1) +
                x * self.U.get_np_array().reshape(-1, 1) +
                y * self.V.get_np_array().reshape(-1, 1))

def raytrace(ray_starting_point, normalized_ray_directions, objects_list, number_of_bounces=1):
    all_objects_intersections = [object.intersect(ray_starting_point, normalized_ray_directions) for object in
                                 objects_list]
    nearest = []

    nearest = functools.reduce(np.minimum, all_objects_intersections)
    color = rgb(0, 0, 0)
    for (object, single_object_intersections) in zip(objects_list, all_objects_intersections):
        hit = (nearest != infinity_plus) & (single_object_intersections == nearest)
        if np.any(hit):
            dc = extract_elements(hit, single_object_intersections)
            Oc = ray_starting_point.extract_elements(hit)
            Dc = normalized_ray_directions.extract_elements(hit)
            cc = object.calculate_intensity(Oc, Dc, dc, objects_list, number_of_bounces)
            color += cc.replace_elements(hit)
    return color

if __name__ == "__main__":
    rgb = clVector3d
    (image_resolution_width, image_resolution_height) = (1000, 800)  # Screen size
    light_position = clPoint3d(5, 5., 5)  # Point light position
    eye_position = clPoint3d(1, 1, -1.)  # Eye position
    look_at_point = clPoint3d(0, 0, 0)
    up = clVector3d(0.0, 1.0, 0.0)
    field_of_view = 90  # Degrees
    infinity_plus = 1.0e39  # an implausibly huge distance
    objects_list = [Sphere(clPoint3d(0.0, 0.0, 0.0), 0.1, rgb(1, 0, 0)),
                    Sphere(clPoint3d(0.0, 0.10, 0.0), 0.1, rgb(0, 1, 0)),
                    Sphere(clPoint3d(0.0, 0.0, 1.0), 0.1, rgb(1, 1, 0)),
                    Sphere(clPoint3d(-0.4, 0.2, 1.0), .1, rgb(.5, .223, .5)),
                    Sphere(clPoint3d(0.0, 0.3, 1.0), .2, rgb(.5, .223, .5)),
                    Sphere(clPoint3d(0.4, 0.4, 1.0), .3, rgb(.5, .223, .5)),
                    Sphere(clPoint3d(0, -99999.5, 0), 99999, rgb(.75, .75, .0), 0.7, texture=1), ]

    # Create a Camera instance
    camera = clCamera(eye_position, look_at_point, up, field_of_view, image_resolution_width, image_resolution_height)

    pixels_world_coordinates = clPoint3d(camera.pixels_world_coordinates[0, :], camera.pixels_world_coordinates[1, :],
                                         camera.pixels_world_coordinates[2, :])

    color = raytrace(eye_position, (pixels_world_coordinates - eye_position).normalize(), objects_list)
    rgb = [Image.fromarray(
        (255 * np.clip(c, 0, 1).reshape((image_resolution_height, image_resolution_width))).astype(np.uint8), "L") for c
        in color.get_components()]
    Image.merge("RGB", rgb).show()  # Image.merge("RGB", rgb).save("fig.png")
    rgb=np.array(rgb)
    print(rgb.shape)
    rgb=np.moveaxis(rgb,0,-1)
    plt.imshow(rgb)
